In [1]:
import pandas as pd
import numpy as np
import shutil
import tempfile
import xlwings as xw
import glob
import re
import regex
from datetime import datetime
import os

from tkinter import *
from tkinter import ttk
from tkinter import filedialog

In [2]:
# Allgemeine Funktionen
def return_error_rows_as_string(error_row):
    error_str = ''
    for e in error_row:
        error_str = error_str + str(e) + ', '
    error_str = error_str[:-2]    
    # Split the string by commas and convert each element to an integer
    numbers_list = [int(num.strip()) for num in error_str.split(',')]
    # Sort the list in ascending order
    numbers_list.sort()
    # Initialize variables
    result = []
    start = numbers_list[0]
    end = numbers_list[0]
    # Iterate over the list and find consecutive numbers
    for i in range(1, len(numbers_list)):
        if numbers_list[i] == end + 1:
            end = numbers_list[i]
        else:
            # Add the range to the result
            if start == end:
                result.append(str(start))
            else:
                result.append(f'{start}-{end}')
            start = numbers_list[i]
            end = numbers_list[i]
    # Add the last range to the result
    if start == end:
        result.append(str(start))
    else:
        result.append(f'{start}-{end}')
    # Join the ranges with commas
    error_str = ', '.join(result)    
    return(error_str)


def zellen_bunt_malen(error_str, column_name, ws, color):
    # get column index from column name
    column_index = vorlage.columns.get_loc(column_name)
    column_letter = xw.utils.col_name(column_index + 1)
    # cell coordinates
    error_row_list = error_str.split(',')
    for error_row in error_row_list:
        if '-' in error_row:
            first_row = error_row.split('-')[0].strip()
            last_row = error_row.split('-')[1].strip()
            cell_coord = column_letter + first_row + ':' + column_letter + last_row
        else:
            cell_coord = column_letter + error_row.strip()
        cell = ws.range(cell_coord)
        cell.color = color

        
def feldname_in_vorlage(feldname, cols_richtig):
    if feldname in cols_richtig:
        feldname_vorlage = cols_richtig[feldname]
    else:
        feldname_vorlage = feldname
    return feldname_vorlage

In [6]:
# Check-Funktionen
felder_liste = ['master_id', 'pat_id', 'sample_id', 'fillcapacity', 'condition', 'aliquoteTypeId',
               'container', 'x', 'y', 'freezeThawCycleNumber', 'freezingDate', 'cbh', 'checkoutCount',
               'comments', 'amountOfDerivatives', 'discharge_quantity', 'discharge_reason',
               'discharge_project', 'discharge_recipient', 'discharge_date', 'discharge_remarks',
               'discharge_cost_unit', 'discharge_expiry_date']

# Check 1: Feldcode-Name
cols_richtig = {}
def check_feldcode(cols, ws, error_count, error_count_total):
    global cols_richtig
    
    cols_richtig = {}
    felder_liste_lower = [entry.lower() for entry in felder_liste]
    
    for col in cols:
        if col not in felder_liste:
            # Suche nach richtiger Schreibweise
            for i in range(len(felder_liste)):
                if col.lower() in felder_liste_lower:
                    index_feld = felder_liste_lower.index(col.lower())
                    col_richtig = felder_liste[index_feld]
                    cols_richtig[col_richtig] = col
                    text_feldcode = Label(frame, text = "Feldcode '{}' ist falsch, richtig wäre '{}'".format(col, col_richtig),
                                            bg = '#eeeee4', font=('Ink free',11))
                    text_feldcode.pack()
                    zellen_bunt_malen('1', col, ws, (220, 20, 60))
                    error_count_total[0] += 1
                    error_count[0] += 1
                    break
                else:
                    if len(col) < 15:
                        match = regex.search(r'({}){}'.format(col.lower(), '{e<3}'), felder_liste_lower[i])
                    else:
                        match = regex.search(r'({}){}'.format(col.lower(), '{e<6}'), felder_liste_lower[i])
                    if match:
                        if ((len(felder_liste[i]) <= len(col) + 2) & (len(felder_liste[i]) >= len(col) - 2)):
                            col_richtig = felder_liste[i]
                            cols_richtig[col_richtig] = col
                            text_feldcode = Label(frame, text = "Feldcode '{}' ist falsch, richtig wäre '{}'".format(col, col_richtig),
                                                    bg = '#eeeee4', font=('Ink free',11))
                            text_feldcode.pack()
                            zellen_bunt_malen('1', col, ws, (220, 20, 60))
                            error_count_total[0] += 1
                            error_count[0] += 1
                            break
                        else:
                            # Leerzeichen vor oder nach dem Wort
                            if len(col) != len(col.strip()):
                                text_feldcode = Label(frame, text = "Feldcode '{}' ist falsch (Leerzeichen zu viel)".format(col),
                                                        bg = '#eeeee4', font=('Ink free',11))
                                text_feldcode.pack()
                                zellen_bunt_malen('1', col, ws, (220, 20, 60))
                                error_count_total[0] += 1
                                error_count[0] += 1
                                break                                
                            else:
                                text_feldcode = Label(frame, text = "Feldcode '{}' ist falsch bzw. nicht in IDA".format(col),
                                                        bg = '#eeeee4', font=('Ink free',11))
                                text_feldcode.pack()
                                zellen_bunt_malen('1', col, ws, (220, 20, 60))
                                error_count_total[0] += 1
                                error_count[0] += 1
                                break
                    # Richtige Schreibweise nicht gefunden
                    else:
                        if i == len(felder_liste)-1:
                            text_feldcode = Label(frame, text = "Feldcode '{}' ist falsch bzw. nicht in IDA".format(col),
                                                    bg = '#eeeee4', font=('Ink free',11))
                            text_feldcode.pack()
                            zellen_bunt_malen('1', col, ws, (220, 20, 60))
                            error_count_total[0] += 1
                            error_count[0] += 1                            
                            

# Check 2: Master ID
def check_masterid(vorlage, ws, error_count, error_count_total):
    master_id_pattern = r'I\d{9}\b'
    error_master_id_row = []
    master_id = feldname_in_vorlage('master_id', cols_richtig)

    for i in range(len(vorlage[master_id])):
        match = re.search(master_id_pattern, str(vorlage[master_id][i]))
        if match:
            continue
        else:
            if str(vorlage[master_id][i]) == 'nan':
                continue
            else:
                error_master_id_row.append(i+2)
    if error_master_id_row != []:
        error_str = return_error_rows_as_string(error_master_id_row)
        text_master = Label(frame, text = "Fehler in Spalte 'master_id' Zeile(n) {}".format(error_str),
                            bg = '#eeeee4', font=('Ink free',11))
        text_master.pack()
        # Zelle in Excel-Datei rot färben
        zellen_bunt_malen(error_str, master_id, ws, (220, 20, 60))
        error_count_total[0] += 1
        error_count[0] += 1
        
        
# Check 3: Sample ID
def check_sampleid(cols, cols_l, vorlage, ws, error_count, error_count_total):
    sample_id_pattern = r'I\d{12}\b'
    sample_id = feldname_in_vorlage('sample_id', cols_richtig)

    error_sample_id_row = []
    for i in range(len(vorlage[sample_id])):
        match = re.search(sample_id_pattern, str(vorlage[sample_id][i]))
        if match:
            continue
        else:
            if str(vorlage[sample_id][i]) == 'nan':
                continue
            else:
                error_sample_id_row.append(i+2)
    if error_sample_id_row != []:
        error_str = return_error_rows_as_string(error_sample_id_row)
        text_sample = Label(frame, text = "Fehler in Spalte 'sample_id' Zeile(n) {}".format(error_str),
                                bg = '#eeeee4', font=('Ink free',11))
        text_sample.pack()
        # Zelle in Excel-Datei rot färben
        zellen_bunt_malen(error_str, sample_id, ws, (220, 20, 60))
        error_count_total[0] += 1
        error_count[0] += 1
        
        
# Check 4: Pat ID
def check_patid(vorlage, ws, error_count, error_count_total):
    error_pat_id_row = []
    pat_id = feldname_in_vorlage('pat_id', cols_richtig)

    # Zuerst wird geprüft, ob die gesamte Spalte im Format np.int64 ist
    if vorlage[pat_id].dtype != np.int64:
        if vorlage[pat_id].dtype == 'object':
            for i in range(len(vorlage[pat_id])):
                if not str(vorlage[pat_id][i]).isdigit():
                    error_pat_id_row.append(i+2)
    if error_pat_id_row != []:
        error_str = return_error_rows_as_string(error_pat_id_row)
        text_pat = Label(frame, text = "(Format-)Fehler in Spalte 'pat_id' Zeile(n) {}".format(error_str),
                        bg = '#eeeee4', font=('Ink free',11))
        text_pat.pack()
        # Zelle in Excel-Datei rot färben
        zellen_bunt_malen(error_str, pat_id, ws, (220, 20, 60))
        error_count_total[0] += 1
        error_count[0] += 1
        
        
# Check 5: Sample und Master ID
def check_sample_master(vorlage, ws, error_count, error_count_total):
    sample_id = feldname_in_vorlage('sample_id', cols_richtig)
    master_id = feldname_in_vorlage('master_id', cols_richtig)
        
    error_sample_master = []
    for i in range(len(vorlage[sample_id])):
        if str(vorlage[sample_id][i]).split('I')[-1][:-3] != str(vorlage[master_id][i]).split('I')[-1]:
            error_sample_master.append(i+2)
    if error_sample_master != []:
        error_str = return_error_rows_as_string(error_sample_master)
        text_pat = Label(frame, text = "'sample_id' stimmt nicht mit 'master_id' überein in Zeile(n) {}".format(error_str),
                        bg = '#eeeee4', font=('Ink free',11))
        text_pat.pack()
        # Zelle in Excel-Datei rot färben
        zellen_bunt_malen(error_str, sample_id, ws, (220, 20, 60))
        zellen_bunt_malen(error_str, master_id, ws, (220, 20, 60))
        error_count_total[0] += 1
        error_count[0] += 1
        
        
# Check 6: Pflichtfeld
def check_pflichtfelder(ws, error_count, error_count_total):
    pflichtfelder = ['master_id', 'pat_id', 'sample_id', 'fillcapacity', 'container',
                     'discharge_quantity', 'discharge_date']
    ausnahme_felder = ['discharge_project', 'discharge_recipient', 'discharge_cost_unit']
    
    for check_col in pflichtfelder:
        check_col = feldname_in_vorlage(check_col, cols_richtig)
        pflichtfeld_row = []
        for i in range(len(vorlage)):
            if str(vorlage[check_col][i]) == 'nan':
                pflichtfeld_row.append(i+2)
        if pflichtfeld_row != []:
            error_str = return_error_rows_as_string(pflichtfeld_row)
            text_pflicht = Label(frame, text = "Pflichtfeld ist leer in Spalte '{}' Zeile(n) {}".format(check_col, error_str),
                                            bg = '#eeeee4', font=('Ink free',11))
            text_pflicht.pack()
            # Zelle in Excel-Datei rot färben
            zellen_bunt_malen(error_str, check_col, ws, (220, 20, 60))
            error_count_total[0] += 1
            error_count[0] += 1
            
    def check_ausnahmefelder(ws, error_count, error_count_total, ausnahme_feld, discharge_nr):
        check_reason = feldname_in_vorlage('discharge_reason', cols_richtig)
        for check_col in ausnahme_felder:
            if check_col == ausnahme_feld:
                check_col = feldname_in_vorlage(check_col, cols_richtig)
                ausnahme_row = []
                for i in range(len(vorlage)):
                    if str(vorlage[check_col][i]) == 'nan':
                        try:
                            if int(vorlage[check_reason][i]) not in discharge_nr:
                                ausnahme_row.append(i+2)
                        except:
                            ausnahme_row.append(i+2)
                if ausnahme_row != []:
                    error_str = return_error_rows_as_string(ausnahme_row)
                    text_pflicht = Label(frame, text = "Pflichtfeld ist leer in Spalte '{}' Zeile(n) {}".format(check_col, error_str),
                                                    bg = '#eeeee4', font=('Ink free',11))
                    text_pflicht.pack()
                    # Zelle in Excel-Datei rot färben
                    zellen_bunt_malen(error_str, check_col, ws, (220, 20, 60))
                    error_count_total[0] += 1
                    error_count[0] += 1
                    
    check_ausnahmefelder(ws, error_count, error_count_total, 'discharge_project', [4, 6, 10, 11, 12])
    check_ausnahmefelder(ws, error_count, error_count_total, 'discharge_recipient', [6, 11, 12])
    check_ausnahmefelder(ws, error_count, error_count_total, 'discharge_cost_unit', [6, 7, 11, 12])
        
            
            
# Check 7: discharge reason
def check_discharge_reason(ws, error_count, error_count_total):
    discharge_row = []
    aliquots_row = []
    capacity_row = []
    container_row = []
    pflichtfelder_aliquots = ['master_id', 'pat_id', 'sample_id', 'fillcapacity', 'condition',
                              'aliquoteTypeId', 'container', 'freezeThawCycleNumber']
    
    discharge_reason = feldname_in_vorlage('discharge_reason', cols_richtig)
    fill_cap = feldname_in_vorlage('fillcapacity', cols_richtig)   
    container = feldname_in_vorlage('container', cols_richtig)
        
    for i in range(len(vorlage)):
        if str(vorlage[discharge_reason][i]) != 'nan':
            try:
                if int(vorlage[discharge_reason][i]) not in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]:
                    discharge_row.append(i+2)
            except:
                discharge_row.append(i+2)
            if vorlage[fill_cap][i] != 0:
                capacity_row.append(i+2)
            if str(vorlage[container][i]) != '99':
                container_row.append(i+2)
        else:
            for check_col in pflichtfelder_aliquots:
                check_col = feldname_in_vorlage(check_col, cols_richtig)   
                if str(vorlage[check_col][i]) == 'nan':
                    aliquots_row.append(i+2)
                    break
                    
    con_row = []
    verbotene_zeichen = ['(', ')', 'box']
    for i in range(len(vorlage)):
        for zeichen in verbotene_zeichen:
            if zeichen in str(vorlage[container][i]).lower():
                if i+2 not in container_row:
                    con_row.append(i+2)
                    break
            
    if aliquots_row != []:
        error_str = return_error_rows_as_string(aliquots_row)
        text_aliquots = Label(frame, text = "Pflichtfeld 'discharge_reason' ist leer in Zeile(n) {}".format(error_str),
                            bg = '#eeeee4', font=('Ink free',11))
        text_aliquots.pack()
        zellen_bunt_malen(error_str, 'discharge_reason', ws, (220, 20, 60))
        error_count_total[0] += 1
        error_count[0] += 1

    if discharge_row != []:
        error_str = return_error_rows_as_string(discharge_row)
        text_dis = Label(frame, text = "Fehler in Spalte 'discharge_reason' Zeile(n) {}".format(error_str),
                        bg = '#eeeee4', font=('Ink free',11))
        text_dis.pack()
        zellen_bunt_malen(error_str, check_col, ws, (220, 20, 60))
        error_count_total[0] += 1
        error_count[0] += 1
                   
    if capacity_row != []:
        error_str = return_error_rows_as_string(capacity_row)
        text_cap = Label(frame, text = "'fillcapacity' ist nicht 0 in Zeile(n) {}".format(error_str),
                        bg = '#eeeee4', font=('Ink free',11))
        text_cap.pack()
        zellen_bunt_malen(error_str, fill_cap, ws, (220, 20, 60))
        error_count_total[0] += 1
        error_count[0] += 1
                   
    if container_row != []:
        error_str = return_error_rows_as_string(container_row)
        text_con = Label(frame, text = "'container' ist nicht 99 in Zeile(n) {}".format(error_str),
                        bg = '#eeeee4', font=('Ink free',11))
        text_con.pack()
        zellen_bunt_malen(error_str, container, ws, (220, 20, 60))
        error_count_total[0] += 1
        error_count[0] += 1
        
    if con_row != []:
        error_str = return_error_rows_as_string(con_row)
        text_con = Label(frame, text = "Fehler in Spalte 'container' Zeile(n) {}".format(error_str),
                          bg = '#eeeee4', font=('Ink free',11))
        text_con.pack()
        # Zelle in Excel-Datei rot färben
        zellen_bunt_malen(error_str, container, ws, (220, 20, 60))
        error_count_total[0] += 1
        error_count[0] += 1 

        
# Check 8: discharge_date
def check_datum(ws, error_count, error_count_total):
    date_col = feldname_in_vorlage('discharge_date', cols_richtig)   
    date_error_row = []
    for i in range(len(vorlage)):
        if str(vorlage[date_col][i]) != 'nan':
            try:
                date_string = str(vorlage[date_col][i])
                date_object = datetime.strptime(date_string, "%d.%m.%Y").date()
            except:
                date_error_row.append(i+2)
    if date_error_row != []:
        error_str = return_error_rows_as_string(date_error_row)
        text_date = Label(frame, text = "Fehler in '{}' Zeile(n) {}".format(date_col, error_str),
                          bg = '#eeeee4', font=('Ink free',11))
        text_date.pack()
        # Zelle in Excel-Datei rot färben
        zellen_bunt_malen(error_str, date_col, ws, (220, 20, 60))
        error_count_total[0] += 1
        error_count[0] += 1
        

# Check 9: discharge_remarks
def check_remarks(ws, error_count, error_count_total):
    rem_col = feldname_in_vorlage('discharge_remarks', cols_richtig)   
    dis_reason = feldname_in_vorlage('discharge_reason', cols_richtig)   
        
    rem_row = []
    for i in range(len(vorlage)):
        if str(vorlage[dis_reason][i]) in ['7', '07', '7.0']:
            if 'neu: ' not in str(vorlage[rem_col][i]):
                rem_row.append(i+2)
                
    if rem_row != []:
        error_str = return_error_rows_as_string(rem_row)
        text_rem = Label(frame, text = "Fehler in 'discharge_remarks' Zeile(n) {}".format(error_str),
                          bg = '#eeeee4', font=('Ink free',11))
        text_rem.pack()
        # Zelle in Excel-Datei rot färben
        zellen_bunt_malen(error_str, rem_col, ws, (220, 20, 60))
        error_count_total[0] += 1
        error_count[0] += 1
        
        
# Check 10: condition und aliquoteTypeId
def check_condition(col, ws, error_count, error_count_total):
    col = feldname_in_vorlage(col, cols_richtig)   
        
    col_row = []
    for i in range(len(vorlage)):
        if str(vorlage[col][i]) != 'nan':
            try:
                int(vorlage[col][i])
            except:
                col_row.append(i+2)
                
    if col_row != []:
        error_str = return_error_rows_as_string(col_row)
        text_col = Label(frame, text = "'{}' ist keine Zahl/Code in Zeile(n) {}".format(col, error_str),
                          bg = '#eeeee4', font=('Ink free',11))
        text_col.pack()
        # Zelle in Excel-Datei rot färben
        zellen_bunt_malen(error_str, col, ws, (220, 20, 60))
        error_count_total[0] += 1
        error_count[0] += 1
        

# Check 11: Datenreihe ausfüllen
def datenreihe(ws, error_count, error_count_total):
    check_spalten = ['discharge_project', 'discharge_cost_unit']
    
    for check_col in check_spalten:
        spalten_row = []
        for i in range(len(vorlage)-1):
            try:
                entry_1 = int(vorlage[check_col][i])
                entry_2 = int(vorlage[check_col][i+1])
            except:
                entry_1 = vorlage[check_col][i]
                entry_2 = vorlage[check_col][i+1]
            try:
                if ('.' in str(entry_1)) & ('.' in str(entry_2)):
                    entry_1 = str(entry_1).split('.')[-1]
                    entry_2 = str(entry_2).split('.')[-1]
                if int(entry_1) == int(entry_2) - 1:
                    spalten_row.append(i+3)
            except Exception as e:
                continue
        if spalten_row != []:
            error_str = return_error_rows_as_string(spalten_row)
            text_col = Label(frame, text = "Bitte überprüfe Datenreihe '{}' in Zeile(n) {}".format(check_col, error_str),
                              bg = '#eeeee4', font=('Ink free',11))
            text_col.pack()
            # Zelle in Excel-Datei grau färben
            zellen_bunt_malen(error_str, check_col, ws, (168, 168, 168))
            error_count_total[0] += 1
            error_count[0] += 1

            
# Check 12: Pflichtfelder bei Versandgründen
def versandgrund(ws, error_count, error_count_total):
    discharge_reason = feldname_in_vorlage('discharge_reason', cols_richtig)    
    discharge_project = feldname_in_vorlage('discharge_project', cols_richtig)
    discharge_recipient = feldname_in_vorlage('discharge_recipient', cols_richtig)        
    discharge_cost_unit = feldname_in_vorlage('discharge_cost_unit', cols_richtig)
    
    project_rows = []
    cost_rows_130 = []
    cost_rows = []
    recipient_rows = []
    aliquot_rows = []
    for i in range(len(vorlage)):
        if str(vorlage[discharge_reason][i]) in ['4', '4.0', '04', '10', '10.0']:
            if str(vorlage[discharge_project][i]) != 'nan':
                project_rows.append(i+2)
            if vorlage[discharge_cost_unit][i] != 130:
                cost_rows_130.append(i+2)
                
        elif str(vorlage[discharge_reason][i]) in ['6', '6.0', '06', '11', '11.0', '12', '12.0']:
            if str(vorlage[discharge_project][i]) != 'nan':
                project_rows.append(i+2)
            if str(vorlage[discharge_recipient][i]) != 'nan':
                recipient_rows.append(i+2)
            if str(vorlage[discharge_cost_unit][i]) != 'nan':
                cost_rows.append(i+2)
                
        elif str(vorlage[discharge_reason][i]) in ['9', '9.0', '09']:
            if vorlage[discharge_cost_unit][i] != 130:
                cost_rows_130.append(i+2)
                
        elif str(vorlage[discharge_reason][i]) in ['7', '7.0', '07']:
            if 'aliquotierung' in str(vorlage[discharge_project][i]).lower():
                aliquot_rows.append(i+2)
            
    if project_rows != []:
        error_str = return_error_rows_as_string(project_rows)
        text_col = Label(frame, text = "'discharge_project' ist bei dem Versandgrund nicht leer in Zeile(n) {}".format(error_str),
                          bg = '#eeeee4', font=('Ink free',11))
        text_col.pack()
        # Zelle in Excel-Datei rot färben
        zellen_bunt_malen(error_str, 'discharge_project', ws, (220, 20, 60))
        error_count_total[0] += 1
        error_count[0] += 1
        
    if recipient_rows != []:
        error_str = return_error_rows_as_string(recipient_rows)
        text_col = Label(frame, text = "'discharge_recipient' ist bei dem Versandgrund nicht leer in Zeile(n) {}".format(error_str),
                          bg = '#eeeee4', font=('Ink free',11))
        text_col.pack()
        # Zelle in Excel-Datei rot färben
        zellen_bunt_malen(error_str, 'discharge_recipient', ws, (220, 20, 60))
        error_count_total[0] += 1
        error_count[0] += 1
        
    if cost_rows != []:
        error_str = return_error_rows_as_string(cost_rows)
        text_col = Label(frame, text = "'discharge_cost_unit' ist bei dem Versandgrund nicht leer in Zeile(n) {}".format(error_str),
                          bg = '#eeeee4', font=('Ink free',11))
        text_col.pack()
        # Zelle in Excel-Datei rot färben
        zellen_bunt_malen(error_str, 'discharge_cost_unit', ws, (220, 20, 60))
        error_count_total[0] += 1
        error_count[0] += 1
        
    if cost_rows_130 != []:
        error_str = return_error_rows_as_string(cost_rows_130)
        text_col = Label(frame, text = "'discharge_cost_unit' ist bei dem Versandgrund nicht 130 in Zeile(n) {}".format(error_str),
                          bg = '#eeeee4', font=('Ink free',11))
        text_col.pack()
        # Zelle in Excel-Datei rot färben
        zellen_bunt_malen(error_str, 'discharge_cost_unit', ws, (220, 20, 60))
        error_count_total[0] += 1
        error_count[0] += 1
        
    if aliquot_rows != []:
        error_str = return_error_rows_as_string(aliquot_rows)
        text_col = Label(frame, text = "In 'discharge_project' steht 'Aliquotierung' in Zeile(n) {}".format(error_str),
                          bg = '#eeeee4', font=('Ink free',11))
        text_col.pack()
        # Zelle in Excel-Datei rot färben
        zellen_bunt_malen(error_str, 'discharge_project', ws, (220, 20, 60))
        error_count_total[0] += 1
        error_count[0] += 1


In [7]:
def go_dodo():
    global frame, vorlage
    # Ladebalken
    progress_bar = ttk.Progressbar(root, orient="horizontal", length=250, mode="determinate")
    progress_bar.grid(row=4, column=1, columnspan=5)
    
    error_count_total = [0]
    
    folder_path = filedialog.askdirectory(title = "Ordner auswählen")
    file_list = os.listdir(folder_path)
    file_list = [file for file in file_list if '$' not in file]
    
    progress_bar.start()   
    
    ### Interface ###    
    root_m = Frame(root, width=900, height=200, bg = '#eeeee4', highlightbackground='#869287',
                   highlightthickness=2)
    root_m.grid(row=7, column=1, columnspan=100)
    # Create a Canvas to hold the Frame and Scrollbars
    canvas = Canvas(root_m, width=800, bg = '#eeeee4')
    canvas.pack(side="left", fill="both", expand=True)

    # Create vertical scrollbar and associate it with the Canvas
    v_scrollbar = Scrollbar(root_m, command=canvas.yview)
    v_scrollbar.pack(side="right", fill="y")
    canvas.configure(yscrollcommand=v_scrollbar.set)

    # Create horizontal scrollbar and associate it with the Canvas
    h_scrollbar = Scrollbar(root_m, command=canvas.xview, orient="horizontal")
    h_scrollbar.pack(side="bottom", fill="x")
    canvas.configure(xscrollcommand=h_scrollbar.set)

    # Create a Frame to hold the label widgets
    frame = Frame(canvas, bg='#eeeee4')
    canvas.create_window((0, 0), window=frame, anchor="nw")
        
    for i in range(len(file_list)):
        error_count = [0]
        vorlage = pd.read_csv(folder_path+'/'+file_list[i], encoding='latin_1', delimiter=';')
        file_name = file_list[i].split('.csv')[0]
        cols = list(vorlage.columns)
        vorlage.to_excel('{}_Check.xlsx'.format(file_name), index=False)
        wb = xw.Book('{}_Check.xlsx'.format(file_name))
        ws = wb.sheets['Sheet1']
        
        filename_text = Label(frame, text = "{}".format(file_name),
                                   bg = '#eeeee4', font=('Ink free',11,'bold'))
        filename_text.pack()
        
        # Checks
        # Feldcode-Namen überprüfen
        check_feldcode(cols, ws, error_count, error_count_total)
        
        # Format von pat_id, master_id und sample_id überprüfen       
        # Spalte master_id
        if ('master_id' in vorlage.columns) | ('master_id' in cols_richtig):
            check_masterid(vorlage, ws, error_count, error_count_total)

        # Spalte sample_id
        cols = list(vorlage.columns)
        cols_l = [col.lower() for col in cols]
        if ('sample_id' in vorlage.columns) | ('sample_id' in cols_richtig):
            check_sampleid(cols, cols_l, vorlage, ws, error_count, error_count_total)

        # Spalte pat_id
        if ('pat_id' in vorlage.columns) | ('pat_id' in cols_richtig):
            check_patid(vorlage, ws, error_count, error_count_total)

        # Check, ob master_id und sample_id identisch sind
        if (('sample_id' in vorlage.columns) | ('sample_id' in cols_richtig)) & (('master_id' in vorlage.columns) | ('master_id' in cols_richtig)):
            check_sample_master(vorlage, ws, error_count, error_count_total)
            
        # leere Pflichtfelder
        check_pflichtfelder(ws, error_count, error_count_total)
        
        # discharge reason
        check_discharge_reason(ws, error_count, error_count_total)
        
        # discharge_date
        check_datum(ws, error_count, error_count_total)
        
        # discharge remarks
        check_remarks(ws, error_count, error_count_total)
        
        # condition
        check_condition('condition', ws, error_count, error_count_total)
        
        # aliquoteTypeId
        check_condition('aliquoteTypeId', ws, error_count, error_count_total)
        
        # Datenreihe
        datenreihe(ws, error_count, error_count_total)
        
        # Pflichfelder bei Versandgrund
        versandgrund(ws, error_count, error_count_total)
        ##################################

        
        error_file = Label(frame, text = 'Anzahl Errors für {}: '.format(file_name) + str(error_count[0]), bg = '#eeeee4', font=('Ink free',9,'bold'))
        error_file.pack()
        error_notifications.append(error_file)
        error_notifications.append(filename_text)
        
        ##### User Interface #####
        error_title = Label(root, text = '\nError-Meldungen: ' + str(error_count_total[0]), bg = '#eeeee4', font=('Ink free',11,'bold'))
        error_title.grid(row=5, column=7, columnspan=15)
        ##########################

        wb.save()
        wb.close()

        step = (100/(len(file_list)+2))
        progress_bar.step(step)
        root.update()
        
    # Feedback, wenn Dateien alle keine Fehler haben
    if (error_count_total == [0]) & (para_flag == False):
        for message in error_notifications:
            message.pack_forget()
            
        error_file = Label(frame, text = 'Alles tip top :)', bg = '#eeeee4', font=('Ink free',15,'bold'))
        error_file.pack()    
        
        
    # Calculate the desired frame width based on the canvas width
    frame_width = canvas.winfo_reqwidth()  # Use canvas width
    # Set the width of the scrollable_frame and prevent resizing
    frame.grid_propagate(False)  # Prevent resizing
    frame.config(width=frame_width)  # Set the width
    # Configure Scrollbar to control scrolling
    frame.update_idletasks()
    canvas.config(scrollregion=canvas.bbox("all"))
        
    progress_bar.stop()
    progress_bar.grid_forget()
        
        
        
    
def go_dodo_solo():
    global frame, vorlage
    # Ladebalken
    progress_bar_solo = ttk.Progressbar(root, orient="horizontal", length=250, mode="determinate")
    progress_bar_solo.grid(row=4, column=1, columnspan=5)
        
    vorlage_path = filedialog.askopenfilename(title = "Datei auswählen")
    progress_bar_solo.start(10)
    
    if vorlage_path not in ['', ' ']:
        vorlage = pd.read_csv(vorlage_path, encoding='latin_1', delimiter=';')
        cols = list(vorlage.columns)
        # neue Datei xxx_Check.xlsx erstellen, um Farbe von fehlerhaften Zellen zu ändern
        file_name = vorlage_path.split('/')[-1].split('.csv')[0]        

        progress_bar_solo.step(5)
        root.update()

            
        vorlage.to_excel('{}_Check.xlsx'.format(file_name), index=False)
        wb = xw.Book('{}_Check.xlsx'.format(file_name))
        ws = wb.sheets['Sheet1']

        progress_bar_solo.step(20)
        root.update()
        ### Interface ###    
        root_m = Frame(root, width=900, height=200, bg = '#eeeee4', highlightbackground='#869287',
                       highlightthickness=2)
        root_m.grid(row=7, column=1, columnspan=100)

        # Create a Canvas to hold the Frame and Scrollbars
        canvas = Canvas(root_m, width=800, bg = '#eeeee4')
        canvas.pack(side="left", fill="both", expand=True)

        # Create vertical scrollbar and associate it with the Canvas
        v_scrollbar = Scrollbar(root_m, command=canvas.yview)
        v_scrollbar.pack(side="right", fill="y")
        canvas.configure(yscrollcommand=v_scrollbar.set)

        # Create horizontal scrollbar and associate it with the Canvas
        h_scrollbar = Scrollbar(root_m, command=canvas.xview, orient="horizontal")
        h_scrollbar.pack(side="bottom", fill="x")
        canvas.configure(xscrollcommand=h_scrollbar.set)

        # Create a Frame to hold the label widgets
        frame = Frame(canvas, bg='#eeeee4')
        canvas.create_window((0, 0), window=frame, anchor="nw")    
    
        error_count = [0]
        error_count_total = [0]
        
        # Checks
        # Feldcode-Namen überprüfen
        check_feldcode(cols, ws, error_count, error_count_total)
        
        # Format von pat_id, master_id und sample_id überprüfen       
        # Spalte master_id
        if ('master_id' in vorlage.columns) | ('master_id' in cols_richtig):
            check_masterid(vorlage, ws, error_count, error_count_total)

        # Spalte sample_id
        cols = list(vorlage.columns)
        cols_l = [col.lower() for col in cols]
        if ('sample_id' in vorlage.columns) | ('sample_id' in cols_richtig):
            check_sampleid(cols, cols_l, vorlage, ws, error_count, error_count_total)

        # Spalte pat_id
        if ('pat_id' in vorlage.columns) | ('pat_id' in cols_richtig):
            check_patid(vorlage, ws, error_count, error_count_total)

        # Check, ob master_id und sample_id identisch sind
        if (('sample_id' in vorlage.columns) | ('sample_id' in cols_richtig)) & (('master_id' in vorlage.columns) | ('master_id' in cols_richtig)):
            check_sample_master(vorlage, ws, error_count, error_count_total)
            
        # leere Pflichtfelder
        check_pflichtfelder(ws, error_count, error_count_total)
        
        # discharge reason
        check_discharge_reason(ws, error_count, error_count_total)
        
        # discharge_date
        check_datum(ws, error_count, error_count_total)
        
        # discharge remarks
        check_remarks(ws, error_count, error_count_total)
        
        # condition
        check_condition('condition', ws, error_count, error_count_total)
        
        # aliquoteTypeId
        check_condition('aliquoteTypeId', ws, error_count, error_count_total)
        
        # Datenreihe
        datenreihe(ws, error_count, error_count_total)
        
        # Pflichfelder bei Versandgrund
        versandgrund(ws, error_count, error_count_total)
        ##################
        
        
        
        # Falls keine Fehler vorhanden sind, Nachrich im GUI
        if (error_count_total == [0]) & ('Parameter' not in vorlage_path):
            null_fehler = Label(frame, text = "Alles tip top :)",
                                    bg = '#eeeee4', font=('Ink free',15))
            null_fehler.pack()

        ##### User Interface #####
        error_title2 = Label(root, text = '\nError-Meldungen: ' + str(error_count[0]), bg = '#eeeee4', font=('Ink free',11,'bold'))
        error_title2.grid(row=5, column=7, columnspan=15)
        ##########################

        wb.save()

        # Calculate the desired frame width based on the canvas width
        frame_width = canvas.winfo_reqwidth()  # Use canvas width
        # Set the width of the scrollable_frame and prevent resizing
        frame.grid_propagate(False)  # Prevent resizing
        frame.config(width=frame_width)  # Set the width
        # Configure Scrollbar to control scrolling
        frame.update_idletasks()
        canvas.config(scrollregion=canvas.bbox("all"))
        
    progress_bar_solo.stop()
    progress_bar_solo.grid_forget()
        

In [8]:
# User Interface
# Basis
root = Tk()
root.title('Versandaustrag-Check')
root.geometry('1000x550')
root.iconbitmap('O:\Forschung & Entwicklung\Allgemein\Vorlagen\Abbildungen\Dodo\dodo_icon.ico')
root.config(bg='#eeeee4')

# alle Buttons auf Startseite
hi = Label(root, text = '\nÜberprüfen der Dateien nach dem Versandaustrag                        \n',
           bg = '#eeeee4', font=('Ink free',15,'bold'))
hi.grid(row=1, column=1, columnspan=30)

browse_text = Label(root, text = 'Bitte wähle den Ordner aus  ', bg = '#eeeee4', font=('Ink free',12))
browse_text.grid(row=2, column=1, columnspan=5)
browse_button = Button(root, text='Browse', font=('Ink free',10,'bold'), bg='#869287',
                       command = go_dodo)
browse_button.grid(row=2, column=6)

browse_solo_text = Label(root, text = 'Oder wähle eine einzelne Datei aus ', bg = '#eeeee4', font=('Ink free',12))
browse_solo_text.grid(row=3, column=1, columnspan=5)
browse_solo_button = Button(root, text='Browse', font=('Ink free',10,'bold'), bg='#869287',
                       command = go_dodo_solo)
browse_solo_button.grid(row=3, column=6)

# Logo
from PIL import ImageTk, Image
frame_logo = Frame(root, width=1, height=1)
frame_logo.grid(row=1, column=0)

img = ImageTk.PhotoImage(Image.open("O:\Forschung & Entwicklung\Allgemein\Vorlagen\Abbildungen\Dodo\dodo-dancing_ohne Hintergrund_ohne Schatten.png").resize((70,70)), master = root)
label = Label(frame_logo, image = img, bg = '#eeeee4')
label.pack()

root.mainloop()